In [1]:
#import packages internal and external
import tests
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from importlib import reload
import pandas as pd
import datasetBuilder
import tools
import scipy
from sklearn.ensemble import HistGradientBoostingClassifier as hgbc
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import RandomForestClassifier as rfc
import sklearn.base
import pickle
import copy
import tests
from sklearn.metrics import roc_auc_score as auc
import os
import plotAndOrderResults

In [2]:
#Set path variables where we want to save created datasets, intermediate outputs, etc
#If you want to run this on your own computer, obviously you should update the path
outputs_path='/Users/jonahpoczobutt/projects/specsim_res/figOutputs'


nist14='/Users/jonahpoczobutt/projects/raw_data/db_csvs/nist14.pkl'
gnps='/Users/jonahpoczobutt/projects/raw_data/db_csvs/gnps.pkl'
mona='/Users/jonahpoczobutt/projects/raw_data/db_csvs/mona_lc.pkl'
metlin='/Users/jonahpoczobutt/projects/raw_data/db_csvs/metlin_experimental.pkl'

#Set the query and target dbs
#these can be different or the same
query = metlin
target = nist14

#This variable toggles whether we do a full run of the notebook, or if we read in variables created in a previous run
fullRun=True

#create directories for results
if fullRun:
    
    os.mkdir(f'{outputs_path}/intermediateOutputs')
    os.mkdir(f'{outputs_path}/fig1')
    os.mkdir(f'{outputs_path}/fig1/fig1a')
    os.mkdir(f'{outputs_path}/fig1/supplementary')
    os.mkdir(f'{outputs_path}/fig1/fig1b')
    os.mkdir(f'{outputs_path}/fig2')


Preprocessing: Creating Target and Matches DFs


In [3]:
if fullRun:

    #This should be replaced with a function to read in all the databases
    query_ = pd.read_pickle(query)
    all_bases = list(set(query_['inchi_base']))

    first_bases = all_bases[:int(len(all_bases)/2)]
    second_bases = all_bases[int(len(all_bases)/2):]

    first_query_ = query_[np.isin(query_['inchi_base'],first_bases)]
    first_query_.reset_index(inplace=True)
    first_query_.to_csv(f'{outputs_path}/intermediateOutputs/first_query.csv')
    del(first_query_)

    second_query_ = query_[np.isin(query_['inchi_base'],first_bases)]
    second_query_.reset_index(inplace=True)
    second_query_.to_csv(f'{outputs_path}/intermediateOutputs/second_query.csv')
    del(second_query_)
    del(query_)

    
    np.save(f'{outputs_path}/intermediateOutputs/first_bases.npy',first_bases)
    np.save(f'{outputs_path}/intermediateOutputs/second_bases.npy',second_bases)
    del(first_bases)
    del(second_bases)


Figure 1a: Global Performance of Individual Metrics/Weighting Schemes

In [4]:
#these are the ppm windows that we want to test
ppm_windows = [10]

#this is the size of the sample we take from the full target
size=1e3

#this is the maximum number of matches we allow for each query, based on the precursor window
max_matches=100

#Similarity methods and transformation parameters below. Leave sim methods as None to run all
noise_threshes=[0.01,0.05,0.1]
centroid_tolerance_vals = [0.05,3]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent',None]
ppm_threshes = [3,5,10,15]
sim_methods=None

if fullRun:
    #we will evaluate the performace of the individual metrics on a large sample from the
    #full target dataset. You can set the size below

    #reload queries and target for individual comparison
    query_=pd.read_pickle(query)
    target_=pd.read_pickle(target)

    for i in ppm_windows:

        matches = datasetBuilder.create_matches_df_new(query_,target_,i,max_matches,size)
        matches.to_pickle(f'{outputs_path}/fig1/supplementary/matches_{i}_ppm.pkl')

        #comparison on large sample
        tests.create_variable_comparisons(
                                noise_threshes=noise_threshes,
                                centroid_threshes=centroid_tolerance_vals,
                                centroid_types=centroid_tolerance_types,
                                powers=powers,
                                sim_methods=sim_methods,
                                matches=matches,
                                outpath = f'{outputs_path}/fig1/fig1a/{i}_ppm.csv',
                                )

Figure 1b: Assessing Metric Stability in Smaller Samples

In [5]:
#this is the size of the sample we take from the full target
size=4e2

#this is the number of times we want to take a sample of the above size
trials=2

if fullRun:
    #we will evaluate the performace of the individual metrics on a large sample from the
    #full target dataset. You can set the size below

    #reload target
    query_=pd.read_pickle(query)
    target_=pd.read_pickle(target)

    for i in range(trials):

        os.mkdir(f'{outputs_path}/fig1/fig1b/{i}')
        
        for j in ppm_windows:

            #create matches after shuffling query
            query_ = query_.sample(frac=1)
            matches = datasetBuilder.create_matches_df_new(query_,target_,j,max_matches,size)

            #comparison on small sample
            tests.create_variable_comparisons(
                                    noise_threshes=noise_threshes,
                                    centroid_threshes=centroid_tolerance_vals,
                                    centroid_types=centroid_tolerance_types,
                                    powers=powers,
                                    sim_methods=sim_methods,
                                    matches=matches,
                                    outpath = f'{outputs_path}/fig1/fig1b/{i}/{j}_ppm.csv'
                                    )

Figures 1a and 1b

In [52]:
reload(plotAndOrderResults)
reload(datasetBuilder)
reload(spectral_similarity)

<module 'spectral_similarity' from '/Users/jonahpoczobutt/projects/SpecSim/spectral_similarity.py'>

In [53]:
#first, generate tables from full size
plotAndOrderResults.fig1a(f'{outputs_path}/fig1/fig1a','/Users/jonahpoczobutt/projects/specsim_res/figOutputs/fig1/supplementary')


max_divergence 0.1 0.05 3.0
yool: ['cosine', 'entropy', 'reverse_dot_product']
yool: ['max_divergence']
oieruwo: Index(['max_divergence_0.1_3.0_0.05da', 'match'], dtype='object')
coluns: Index(['mass_reduction_target', 'ent_query_0.01_None_0.05da',
       'npeaks_query_0.01_None_0.05da', 'normalent_query_0.01_None_0.05da',
       'mass_reduction_query_0.01_None_0.05da', 'ent_target_0.01_None_0.05da',
       'npeaks_target_0.01_None_0.05da', 'normalent_target_0.01_None_0.05da',
       'mass_reduction_target_0.01_None_0.05da', 'cosine_0.01_None_0.05da',
       'entropy_0.01_None_0.05da', 'reverse_dot_product_0.01_None_0.05da',
       'max_divergence_0.1_3.0_0.05da', 'match'],
      dtype='object')
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jonahpoczobutt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/5m/x744wd9118v_slyjdnmgqr0r0000gn/T/ipykernel_76491/1336158449.py", line 2, in <module>
    plotAndOrderResults.fig1a(f'{outputs_path}/fig1/fig1a','/Users/jonahpoczobutt/projects/specsim_res/figOutputs/fig1/supplementary')
  File "/Users/jonahpoczobutt/projects/SpecSim/plotAndOrderResults.py", line 57, in fig1a
    names, xs, ys = plot1a_data(orig_res)
                    ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonahpoczobutt/projects/SpecSim/plotAndOrderResults.py", line 97, in plot1a_data
    xs_[j]=1- (running_neg/tot_false) #1-TNR
    ^^^
NameError: name 'xs_' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jonahpoczobutt/anaconda3/lib/python3.11/site-packages/IPytho

In [14]:
pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/figOutputs/fig1/fig1a/10_ppm.csv', header=None)

,0,1,2,3
0,0,max_manhattan,0.667100,0.01_0.05_da_0.25
1,1,hellinger,0.717839,0.01_0.05_da_0.25
2,2,max_absolute_value,0.667100,0.01_0.05_da_0.25
3,3,entropy,0.705250,0.01_0.05_da_0.25
4,4,lorentzian_jonah,0.705556,0.01_0.05_da_0.25
...,...,...,...,...
2935,93,max_common_mass,0.493896,0.1_3_ppm_None
2936,94,clark,0.478212,0.1_3_ppm_None
2937,95,max_l2,0.493920,0.1_3_ppm_None
2938,96,probabilistic_symmetric_chi_squared,0.475150,0.1_3_ppm_None


In [10]:
#figure 1
#column 3 is noise clip, peak consolidation value and type, weighting
#none weighting is the original weighted entropy scheme from the paper
ppm3 = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/individual_sims/matches_3_ppm.csv', header=None)
ppm3.sort_values(by=2, ascending=False, inplace=True)

ppm5 = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/individual_sims/matches_5_ppm.csv', header=None)
ppm5.sort_values(by=2, ascending=False, inplace=True)

ppm10 = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/individual_sims/matches_10_ppm.csv', header=None)
ppm10.sort_values(by=2, ascending=False, inplace=True)

ppm15 = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/individual_sims/matches_15_ppm.csv', header=None)
ppm15.sort_values(by=2, ascending=False, inplace=True)